In [ ]:
!pip install fasttext

     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3092228 sha256=1982c1c77bcd812390459303bdcdabf9546b346e717442a14103e12631458f80
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import pandas as pd

In [ ]:
df_en = pd.read_csv('/content/en_ewt-ud-train.conllu', delimiter='\t', comment='#', 
                 names=['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC'],
                 skip_blank_lines=False, error_bad_lines=False, quoting=3) #ignore quote to read the " character
df_en['LEMMA'] = df_en['LEMMA'].fillna(value='end_line_word')
# df_en = df_en.drop(['ID'], axis=1)

In [ ]:
len(df_en['LEMMA'].unique())

14712

In [ ]:
df_en['UPOS'].unique()

array(['PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'DET', 'ADP', nan, 'AUX',
       'PRON', 'PART', 'SCONJ', 'NUM', 'ADV', 'CCONJ', '_', '``', "''",
       'X', 'INTJ', 'SYM', 'NN', ',', 'NNS'], dtype=object)

In [ ]:
dict_pos = {
    "PROPN":"properNoun", 
    "PUNCT":"punctuation", 
    "ADJ":"adjective", 
    "NOUN":"noun", 
    "VERB":"verb", 
    "DET":"determiner", 
    "ADP":"adposition", 
    "AUX":"auxiliary", 
    "PRON":"pronoun", 
    "PART":"particle", 
    "SCONJ":"subordinatingConjunction", 
    "NUM":"number", 
    "ADV":"adverb", 
    "CCONJ":"coordinatingConjunction", 
    "INTJ":"interjection",
    "SYM":"symbol"
}
list_dict_key = list(dict_pos.keys())
def pos_lemonpos(x):
  if x not in list_dict_key:
    return 'none'
  else:
    return dict_pos[x]

In [ ]:
df_en['pos'] = df_en['UPOS'].apply(lambda x : pos_lemonpos(x))
df_en['word_form'] = df_en['LEMMA'] + '_' + df_en['pos'] + '_en'

In [ ]:
def df_to_sentence(df, lang='en'):
  lemma = df['LEMMA']
  pos = df['pos']
  word_merge = lemma + '_' + pos + '_' + lang
  sentence = ' '.join(word_merge)
  sentence = sentence.replace('end_line_word_none_en', '\n')
  sentence = sentence.replace('\n ', '\n')
  return sentence

In [ ]:
sentence = df_to_sentence(df_en, lang='en')

In [ ]:
# text_file = open("Output.txt", "w")
# text_file.write("%s" % sentence)
# text_file.close()

In [ ]:
!wget https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-3424/allzip

--2021-04-28 11:45:59--  https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-3424/allzip
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘allzip’

allzip                  [        <=>         ] 480.01M  10.5MB/s    in 51s     

2021-04-28 11:46:54 (9.36 MB/s) - ‘allzip’ saved [503323392]



In [ ]:
!unzip allzip

Archive:  allzip
  inflating: ud-treebanks-v2.7.tgz   
  inflating: ud-documentation-v2.7.tgz  
  inflating: ud-tools-v2.7.tgz       


In [ ]:
!tar -xzvf ud-treebanks-v2.7.tgz

In [ ]:
import os
path = '/content/ud-treebanks-v2.7/'
list_pd = os.listdir(path)
list_en = [x for x in list_pd if 'UD_English' in x and 'GUMReddit' not in x and 'English-ESL' not in x]
list_es = [x for x in list_pd if 'UD_Spanish' in x]
list_fr = [x for x in list_pd if 'UD_French' in x]

In [ ]:
dict_lang = {"en":list_en, "es":list_es, "fr":list_fr}

In [ ]:
def get_all_data(dfs, lang):
  sens = []
  for m_path_df in dfs: 
    df = pd.read_csv(m_path_df, delimiter='	', comment='#', 
                    names=['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC'],
                    skip_blank_lines=False, quoting=3) #ignore quote to read the " character
    df['LEMMA'] = df['LEMMA'].fillna(value='end_line_word')
    df = df.drop(['ID'], axis=1)
    df['pos'] = df['UPOS'].apply(lambda x : pos_lemonpos(x))
    sentence = df_to_sentence(df, lang=lang)
    sens.append(sentence)
  return sens
def to_file(sens, fileName = 'file.txt'):
  with open(fileName, "w") as f:
    for item in sens:
      f.write("%s\n" % item)\

In [ ]:
for key in list(dict_lang.keys()):
  list_lang = dict_lang[key]
  dfs_path = []
  for folder in list_lang:
    folder_path = path+folder
    m_files = [x for x in os.listdir(folder_path) if '.conllu' in x]
    for m_file in m_files:
      dfs_path.append(path+folder+'/'+m_file)
  sens_lang = get_all_data(dfs_path, key)
  to_file(sens_lang, fileName=key+"_corpus.txt")

In [ ]:
import fasttext
model = fasttext.train_unsupervised(input='/content/EN_corpus.txt', minCount=1, ws=3, epoch=50) 

In [ ]:
words = model.get_words()

In [ ]:
model.

26291